## Práctica: Exámen año pasado
### Daniel Corral Ruiz
### Máster Data Science para Finanzas CUNEF

## 1. The World Bank's international debt data
<p>No es que los humanos solo tengamos deudas para administrar nuestras necesidades. Un país también puede endeudarse para administrar su economía. Por ejemplo, el gasto en infraestructura es un ingrediente costoso requerido para que los ciudadanos de un país lleven una vida cómoda. El Banco Mundial es la organización que proporciona deuda a los países.</p>

<!-- <p>En este notebook, vamos a analizar los datos de la deuda internacional recopilados por el Banco Mundial. El conjunto de datos contiene información sobre el monto de la deuda (en USD) que deben los países en desarrollo en varias categorías.</p>  -->
    

#### 1. Inicializar y cargar el contexto spark

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
!tar xf spark-2.4.7-bin-hadoop2.7.tgz #d
!pip install -q findspark 

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark import SparkContext
sc = SparkContext()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

##### 2. Número de países distintos en cada dataset. Coinciden?

Comenzamos subiendo ambos archivos a nuestro colab, mediante el símbolo de la carpeta. Para ello hemos eliminado el nombre de las variables para poder estudiar los números de países distintos que encontramos en ambos datasets.

In [ ]:
desarrollo = "indicadores_desarrollo.csv" # subimos el archivo
desarrollo = sc.textFile(desarrollo) 
primera_fila = desarrollo.take(1)[0] 
desarrollo = desarrollo.filter(lambda x: x != primera_fila) #Nos quedamos con todo lo que no sea la primera fila
desarrollofinal = desarrollo.map(lambda x: x.split(",")) # separamos por comas
desarrollofinal.take(10)

[['Albania',
  'ALB',
  '2014 [YR2014]',
  '1023040800000',
  '73.3202469198429',
  '12.259'],
 ['Albania',
  'ALB',
  '2015 [YR2015]',
  '1145500940000',
  '79.8644448747259',
  '12.197'],
 ['Albania',
  'ALB',
  '2016 [YR2016]',
  '1188814490000',
  '80.7355748396923',
  '12.08'],
 ['Australia',
  'AUS',
  '2014 [YR2014]',
  '674700000',
  '0.042207528166503',
  '13.2'],
 ['Australia',
  'AUS',
  '2015 [YR2015]',
  '762718000',
  '0.0469480198522591',
  '12.9'],
 ['Australia',
  'AUS',
  '2016 [YR2016]',
  '907555000',
  '0.054648482520169',
  '12.9'],
 ['"Bahamas',
  ' The"',
  'BHS',
  '2014 [YR2014]',
  '5158420000',
  '47.2672793747079',
  '14.34'],
 ['"Bahamas',
  ' The"',
  'BHS',
  '2015 [YR2015]',
  '5655150000',
  '48.1195170307089',
  '14.224'],
 ['"Bahamas',
  ' The"',
  'BHS',
  '2016 [YR2016]',
  '6138814000',
  '51.4207431481606',
  '14.131'],
 ['Barbados',
  'BRB',
  '2014 [YR2014]',
  '12175704025',
  '129.629591844133',
  '10.916']]

Observamos un total de 51 países diferentes.

In [ ]:
desarrollopaises = desarrollofinal.map(lambda x: (x[0]))
desarrollopaises.distinct().count()

51

Realizamos el mismo procedimiento con el dataset de los indicadores_deuda.

In [ ]:
deuda = "indicadores_deuda.csv"
deuda = sc.textFile(deuda)
primera_fila = deuda.take(1)[0]
deuda = deuda.filter(lambda x: x != primera_fila) #Nos quedamos con todo lo que no sea la primera fila
deudafinal = deuda.map(lambda x: x.split(","))
deudafinal.take(3)

[['Afghanistan',
  'AFG',
  '61739336.9',
  '100847181.9',
  '',
  '39107845',
  '100847181.9',
  '',
  '',
  '',
  '',
  '49114729.4',
  '72894453.7',
  '23779724.3',
  '72894453.7',
  '',
  '',
  '',
  '39903620.1',
  '53239440.1',
  '',
  '13335820',
  '53239440.1',
  '',
  '',
  '',
  ''],
 ['Albania',
  'ALB',
  '54250280.6',
  '790248675.2',
  '514185620',
  '182197616.7',
  '236447897.3',
  '0',
  '39445139.5',
  '170018.4',
  '39615157.9',
  '76050616.1',
  '317194512.5',
  '234321242.3',
  '310371858.4',
  '4542664.9',
  '2279989.2',
  '6822654.1',
  '13847333.6',
  '165602386.9',
  '87884000',
  '28101536.1',
  '41948869.7',
  '31030688.2',
  '4618504.3',
  '120324.7',
  '35769517.2'],
 ['Algeria',
  'DZA',
  '95188724.6',
  '171185188.1',
  '75420000',
  '0',
  '95188724.6',
  '',
  '576463.5',
  '0',
  '576463.5',
  '10320772.2',
  '10320772.2',
  '',
  '10320772.2',
  '',
  '',
  '',
  '7680627.6',
  '19031728.7',
  '3220000',
  '8094779',
  '15775406.6',
  '',
  '13192.3'

Observamos un total de 123 países distintos.

In [ ]:
deudapaises = deudafinal.map(lambda x: (x[0]))
deudapaises.distinct().count()

123

Respondiendo a la pregunta inicial, respondemos que no existen el mismo número de países.

##### 3. Total de deuda contraida por cada pais: total amount of debt (in USD) DT.AMT.MLAT.CD

Escogemos ambas columnas para su estudio, el nombre del país y la variable a estudiar.

In [ ]:
deuda_pais = deudafinal.map(lambda x: (x[0], x[5]))
deuda_pais.take(100)

[('Afghanistan', '39107845'),
 ('Albania', '182197616.7'),
 ('Algeria', '0'),
 ('Angola', '124688691.9'),
 ('Armenia', '94331207.1'),
 ('Azerbaijan', '513623656.5'),
 ('Bangladesh', '1039564682.7'),
 ('Belarus', '595438826'),
 ('Belize', '29683326.5'),
 ('Benin', '91631639.5'),
 ('Bhutan', '17390420.5'),
 ('Bolivia', '357721211.5'),
 ('Bosnia and Herzegovina', '290170946.5'),
 ('Botswana', '125652344.5'),
 ('Brazil', '2514318741.6'),
 ('Bulgaria', '228993019.8'),
 ('Burkina Faso', '119510893'),
 ('Burundi', '16115111.8'),
 ('Cabo Verde', '39970124.9'),
 ('Cambodia', '72307001.8'),
 ('Cameroon', '152095968.1'),
 ('Central African Republic', '2241813.8'),
 ('Chad', '95738709.5'),
 ('China', '2615723714.1'),
 ('Colombia', '1100907553.6'),
 ('Comoros', '2182053.1'),
 ('"Congo', ''),
 ('"Congo', ''),
 ('Costa Rica', '272808260.7'),
 ("Cote d'Ivoire", '84629406.5'),
 ('Djibouti', '34254909.2'),
 ('Dominica', '11130355.8'),
 ('Dominican Republic', '254728306.1'),
 ('Ecuador', '578894793.9'),


##### 4. Media de los indicadores de deuda (tabla uno): DT.AMT.BLAT.CD, DT.DIS.BLAT.CD, DT.INT.BLAT.CD

Seleccionamos las variables a estudiar y realizamos la media para variable.

In [ ]:
indicador1 = deudafinal.map(lambda x: ( x[2]))
indicador2 = deudafinal.map(lambda x: ( x[11]))
indicador3 = deudafinal.map(lambda x: ( x[18]))

Media de DT.AMT.BLAT.CD

In [ ]:
indicador1=indicador1.filter(lambda x_int: x_int.isdigit())
indicador1=indicador1.map(lambda x: int(x))
indicador1.mean()

61416650.11111111

Media  DT.DIS.BLAT.CD

In [ ]:
indicador2=indicador2.filter(lambda x_int: x_int.isdigit())
indicador2=indicador2.map(lambda x: int(x))
indicador2.mean()

506624979.33333343

Media DT.INT.BLAT.CD

In [ ]:
indicador3=indicador3.filter(lambda x_int: x_int.isdigit())
indicador3=indicador3.map(lambda x: int(x))
indicador3.mean()

53762351.333333336

##### 5. Los 20 paises con DT.AMT.DLXF.CD más alto

Elegimos las variables a escoger, el nombre del país y DT.AMT.DLXF.CD, y mediante la funcion top escogemos los 20 países con DT.AMT.DLXF.CD más alto.

In [ ]:
top20 = deudafinal.map(lambda x: (float(x[3]),x[0]))
top20.top(20)

[(96218620835.7, 'China'),
 (90041840304.1, 'Brazil'),
 (66589761833.5, 'Russian Federation'),
 (51555031005.8, 'Turkey'),
 (48756295898.2, 'South Asia'),
 (31923507000.8, 'India'),
 (30916112653.8, 'Indonesia'),
 (27482093686.4, 'Kazakhstan'),
 (25218503927.0, 'Mexico'),
 (25197029299.4, 'Least developed countries: UN classification'),
 (20483289208.0, 'IDA only'),
 (14013783350.4, 'Romania'),
 (11985674438.7, 'Colombia'),
 (11067045628.1, 'Angola'),
 (10404814960.2, 'Cameroon'),
 (9506919669.6, 'Lebanon'),
 (9474257551.9, 'South Africa'),
 (8873505909.2, 'Vietnam'),
 (8336013891.3, 'Pakistan'),
 (8148995625.6, 'Ukraine')]

##### 6. Pais con los datos informados todos los años.

Ponemos un 1 en cada país durante los distintos años que tengamos, en el caso de que un país tenga todos los años el resultado será 4, mientras que si falta algún año sera 3,2,1,0. Realizamos un filtro en el que solo se escoja aquellos en los que el resultado haya sido 4, es decir, que tengan todos los años información. En nuestro caso solo presenta Malawi datos todos los años.

In [ ]:
todos_años = desarrollofinal.map(lambda x: (x[0],1)).reduceByKey(lambda x,y : x+y)
todos_años = todos_años.filter(lambda x: x[1] == 4)
todos_años.take(10)

[('Malawi', 4)]

##### 7. Media anual de los distintos indicadores de desarrollo

Escogemos primera variable para cada año:

In [ ]:
desarrollo1_2014 = desarrollofinal.filter(lambda x: x[2] == '2014 [YR2014]')
desarrollo1_2014 = desarrollo1_2014.map(lambda x: x[3])
desarrollo1_2014=desarrollo1_2014.filter(lambda x_int: x_int.isdigit())
desarrollo1_2014=desarrollo1_2014.map(lambda x: int(x))
desarrollo1_2014.mean()

116610529707695.64

In [ ]:
desarrollo1_2015 = desarrollofinal.filter(lambda x: x[2] == '2015 [YR2015]')
desarrollo1_2015 = desarrollo1_2015.map(lambda x: x[3])
desarrollo1_2015=desarrollo1_2015.filter(lambda x_int: x_int.isdigit())
desarrollo1_2015=desarrollo1_2015.map(lambda x: int(x))
desarrollo1_2015.mean()

184827281799554.75

In [ ]:
desarrollo1_2016 = desarrollofinal.filter(lambda x: x[2] == '2016 [YR2016]')
desarrollo1_2016 = desarrollo1_2016.map(lambda x: x[3])
desarrollo1_2016=desarrollo1_2016.filter(lambda x_int: x_int.isdigit())
desarrollo1_2016=desarrollo1_2016.map(lambda x: int(x))
desarrollo1_2016.mean()

274875240137558.72

In [ ]:
desarrollo1_2017 = desarrollofinal.filter(lambda x: x[2] == '2017 [YR2017]')
desarrollo1_2017 = desarrollo1_2017.map(lambda x: x[3])
desarrollo1_2017.take(1)

['2239858619962.69']

Escogemos segunda variable:

In [ ]:
desarrollo2_2014 = desarrollofinal.filter(lambda x: x[2] == '2014 [YR2014]')
desarrollo2_2014 = desarrollo2_2014.map(lambda x: x[4])
desarrollo2_2014=desarrollo2_2014.map(lambda x: float(x))
desarrollo2_2014.mean()

58.15214676864467

In [ ]:
desarrollo2_2015 = desarrollofinal.filter(lambda x: x[2] == '2015 [YR2015]')
desarrollo2_2015 = desarrollo2_2015.map(lambda x: x[4])
desarrollo2_2015=desarrollo2_2015.map(lambda x: float(x))
desarrollo2_2015.mean()

59.76292583345264

In [ ]:
desarrollo2_2016 = desarrollofinal.filter(lambda x: x[2] == '2016 [YR2016]')
desarrollo2_2016 = desarrollo2_2016.map(lambda x: x[4])
desarrollo2_2016=desarrollo2_2016.map(lambda x: float(x))
desarrollo2_2016.mean()

60.44188301621869

In [ ]:
desarrollo2_2017 = desarrollofinal.filter(lambda x: x[2] == '2017 [YR2017]')
desarrollo2_2017 = desarrollo2_2017.map(lambda x: x[4])
desarrollo2_2017.take(10)

['48.659564238322']

Escogemos ahora la última variable:



In [ ]:
desarrollo3_2014 = desarrollofinal.filter(lambda x: x[2] == '2014 [YR2014]')
desarrollo3_2014 = desarrollo3_2014.map(lambda x: x[5])
desarrollo3_2014=desarrollo3_2014.filter(lambda x_int: x_int.isdigit())
desarrollo3_2014=desarrollo3_2014.map(lambda x: int(x))
desarrollo3_2014.mean()

12.333333333333334

In [ ]:
desarrollo3_2015 = desarrollofinal.filter(lambda x: x[2] == '2015 [YR2015]')
desarrollo3_2015 = desarrollo3_2015.map(lambda x: x[5])
desarrollo3_2015=desarrollo3_2015.filter(lambda x_int: x_int.isdigit())
desarrollo3_2015=desarrollo3_2015.map(lambda x: int(x))
desarrollo3_2015.mean()

11.333333333333334

In [ ]:
desarrollo3_2016 = desarrollofinal.filter(lambda x: x[2] == '2016 [YR2016]')
desarrollo3_2016 = desarrollo3_2016.map(lambda x: x[5])
desarrollo3_2016=desarrollo3_2016.filter(lambda x_int: x_int.isdigit())
desarrollo3_2016=desarrollo3_2016.map(lambda x: int(x))
desarrollo3_2016.mean()

19.0

In [ ]:
desarrollo3_2017 = desarrollofinal.filter(lambda x: x[2] == '2017 [YR2017]')
desarrollo3_2017 = desarrollo3_2017.map(lambda x: x[5])
desarrollo3_2017.take(10)

['34.593']

##### 8. Podrías decirme el total de deuda acumulada DT.AMT.MLAT.CD por los 10 países con un valor en media menor de SP.DYN.CBRT.IN

En primer lugar calculamos la media de SP.DYN.CBRT.IN:

In [ ]:
mediamenor = desarrollofinal.map(lambda x: x[5])
mediamenor=mediamenor.filter(lambda x_int: x_int.isdigit())
mediamenor=mediamenor.map(lambda x: int(x))
mediamenor.mean()

13.625

Filtramos los paises de menor a mayor para ver los 10 países con el valor más pequeño que la deuda:

In [ ]:
deudamenor = desarrollofinal.filter(lambda x: x[2] == '2016 [YR2016]')
natalidad_pais = deudamenor.map(lambda x: (float(x[5]),x[0]))
natalidad_pais.takeOrdered(10, key = lambda x: x[0])

[(7.8, 'Japan'),
 (7.9, 'San Marino'),
 (8.8, 'Spain'),
 (9.4, 'Singapore'),
 (9.7, 'Hungary'),
 (10.3, 'Ukraine'),
 (10.445, 'Moldova'),
 (10.5, 'Switzerland'),
 (11.8, 'United Kingdom'),
 (12.0, 'Palau')]

Podemos observar como unicamente dos países tienen el total de deuda acumulada DT.AMT.MLAT.CD, que son Palau y Ukraine.

In [ ]:
Palau = deudafinal.filter(lambda x: 'Palau' in x[0])
natalidad_deuda = Palau.map(lambda x: (float(x[5]), x[0]))
natalidad_deuda.take(1)

[]

In [ ]:
Ukraine = deudafinal.filter(lambda x: 'Ukraine' in x[0])
natalidad_deuda = Ukraine.map(lambda x: (float(x[5]), x[0]))
natalidad_deuda.take(1)

[(1072472830.3, 'Ukraine')]

In [ ]:
Moldova = deudafinal.filter(lambda x: 'Moldova' in x[0])
natalidad_deuda = Moldova.map(lambda x: (float(x[5]), x[0]))
natalidad_deuda.take(1)

[(135842865.5, 'Moldova')]

##### 9. ¿Hay alguna relación entre los nacimientos y el indicador DT.AMT.DLXF.CD? ¿Cómo podrías demostrarlo?